NAME: ICFES <br>
DESCRIPTION: data import and cleaning of ICFES administrative data of their standarized test in Colombia<br>
PROJECT: Childhood in Barranquilla - Econometria Consultores<br>
AUTHOR: Juan José Rincón  <br>

In [1]:
import pandas as pd
import numpy as np
import os
from re import sub
from sklearn import preprocessing
from statistics import mean, stdev

In [2]:
pd.options.display.max_columns = 200

In [3]:
pd.options.display.max_rows = 4000

In [4]:
os.chdir("/Users/JuanJose/Library/CloudStorage/GoogleDrive-j.rincon@econometria.com/Mi unidad/Libro Niñez DS")


Se arregla un archivo que tiene un formato diferente, para poderlo incluir en el loop de manera facil

In [5]:
p2015_1 =  pd.read_csv('Resultados Saber11/SB11_2015_1.csv',delimiter=',', low_memory=False)
p2015_1.columns = list(map(str.upper,p2015_1.columns))
p2015_1.to_csv('Resultados Saber11/SB11_20151.csv',sep=';')

Se carga la base de datos de la población de 5-16 años por municipio por año 

In [6]:
MEN_Poblacion_5_16 = pd.read_csv("datos educacion media/MEN_Poblacion_5_16.csv")

Se establecen los directorios y la estructura de las bases de datos

In [7]:
Icfes = {'p2011_1': {'dir':'Resultados Saber11/SB11_20111.csv','df':[],"AÑO":2011},
         'p2011_2': {'dir':'Resultados Saber11/SB11_20112.csv','df':[],"AÑO":2011},
         'p2012_1': {'dir':'Resultados Saber11/SB11_20121.csv','df':[],"AÑO":2012},
         'p2012_2': {'dir':'Resultados Saber11/SB11_20122.csv','df':[],"AÑO":2012},
         'p2013_1': {'dir':'Resultados Saber11/SB11_20131.csv','df':[],"AÑO":2013},
         'p2013_2': {'dir':'Resultados Saber11/SB11_20132.csv','df':[],"AÑO":2013},
         'p2014_1': {'dir':'Resultados Saber11/SB11_20141.csv','df':[],"AÑO":2014},
         'p2014_2': {'dir':'Resultados Saber11/SB11_20142.csv','df':[],"AÑO":2014},
         'p2015_1': {'dir':'Resultados Saber11/SB11_20151.csv','df':[],"AÑO":2015},
         'p2015_2': {'dir':'Resultados Saber11/SB11_20152.csv','df':[],"AÑO":2015},
         'p2016_1': {'dir':'Resultados Saber11/SB11_20161.csv','df':[],"AÑO":2016},
         'p2016_2': {'dir':'Resultados Saber11/SB11_20162.csv','df':[],"AÑO":2016},
         'p2017_1': {'dir':'Resultados Saber11/SB11_20171.csv','df':[],"AÑO":2017},
         'p2017_2': {'dir':'Resultados Saber11/SB11_20172.csv','df':[],"AÑO":2017},
         'p2018_1': {'dir':'Resultados Saber11/SB11_20181.csv','df':[],"AÑO":2018},
         'p2018_2': {'dir':'Resultados Saber11/SB11_20182.csv','df':[],"AÑO":2018},
         'p2019': {'dir':'Resultados Saber11/SB11_20192.csv','df':[],"AÑO":2019},
         'p2020_1': {'dir':'Resultados Saber11/SB11_20201.csv','df':[],"AÑO":2020},
         'p2020_2': {'dir':'Resultados Saber11/SB11_20202.csv','df':[],"AÑO":2020},
         'p2021_1': {'dir':'Resultados Saber11/SB11_20211.csv','df':[],"AÑO":2021},
         'p2021_2': {'dir':'Resultados Saber11/SB11_20212.csv','df':[],"AÑO":2021},
        }

Se importan y limpian los datos en dos loops, 1 para los resultados del icfes antes del cambio del puntaje global en el 2014-2, y el 2 para los resultados despues de este cambio

In [8]:
for key in Icfes.keys():
    if key in ['p2011_1','p2011_2','p2012_1','p2012_2','p2013_1','p2013_2','p2014_1']:
        Icfes[key]["df"] =  pd.read_csv(
            Icfes[key]["dir"],delimiter=';',
            usecols= ["COLE_DEPTO_UBICACION","COLE_COD_DEPTO_UBICACION","COLE_MCPIO_UBICACION",
                      "COLE_COD_MCPIO_UBICACION",'PUNT_MATEMATICAS','PUNT_INGLES','PUNT_CIENCIAS_SOCIALES',
                      'PUNT_BIOLOGIA','PUNT_FILOSOFIA','PUNT_FISICA','PUNT_QUIMICA','PUNT_LENGUAJE',
                      'COLE_NATURALEZA','COLE_JORNADA',"ESTU_COD_RESIDE_MCPIO"],
            dtype = {"COLE_DEPTO_UBICACION":"str","COLE_MCPIO_UBICACION":"str",'PUNT_MATEMATICAS':"str",
                     'PUNT_INGLES':"str",'PUNT_CIENCIAS_SOCIALES':"str",'PUNT_BIOLOGIA':"str",
                     'PUNT_FILOSOFIA':"str",'PUNT_FISICA':"str",'PUNT_QUIMICA':"str",'PUNT_LENGUAJE':"str",
                     'COLE_NATURALEZA':"str",'COLE_JORNADA':"str"},low_memory=False )
        
        Icfes[key]["df"].columns = list(map(str.upper,Icfes[key]["df"].columns))
        
        Icfes[key]["df"].rename(columns = {'COLE_DEPTO_UBICACION':'DEPTO','COLE_MCPIO_UBICACION':'MUNICIPIO',
                                           "COLE_COD_DEPTO_UBICACION":'COD_DEPTO',
                                           'ESTU_COD_RESIDE_MCPIO':'COD_MUN_ESTU',
                                           'COLE_COD_MCPIO_UBICACION':'COD_MUN_COLE'}, inplace = True)
        
        Icfes[key]["df"] = Icfes[key]["df"].assign(
            PUNT_MATEMATICAS = pd.to_numeric(Icfes[key]["df"].PUNT_MATEMATICAS.str.replace(',', '.')),
            PUNT_LENGUAJE = pd.to_numeric(Icfes[key]["df"].PUNT_LENGUAJE.str.replace(',', '.')),
            PUNT_INGLES = pd.to_numeric(Icfes[key]["df"].PUNT_INGLES.str.replace(',', '.')),
            PUNT_CIENCIAS_SOCIALES = 
            pd.to_numeric(Icfes[key]["df"].PUNT_CIENCIAS_SOCIALES.str.replace(',', '.')),
            PUNT_FISICA = pd.to_numeric(Icfes[key]["df"].PUNT_FISICA.str.replace(',', '.')),
            PUNT_QUIMICA = pd.to_numeric(Icfes[key]["df"].PUNT_QUIMICA.str.replace(',', '.')),
            PUNT_BIOLOGIA = pd.to_numeric(Icfes[key]["df"].PUNT_BIOLOGIA.str.replace(',', '.')),
            PUNT_FILOSOFIA = pd.to_numeric(Icfes[key]["df"].PUNT_FILOSOFIA.str.replace(',', '.')) )
        
        Icfes[key]["df"] = Icfes[key]["df"].assign(
            PUNT_MATEMATICAS_STD = preprocessing.scale(Icfes[key]["df"].PUNT_MATEMATICAS),
            PUNT_LENGUAJE_STD = preprocessing.scale(Icfes[key]["df"].PUNT_LENGUAJE),
            PUNT_INGLES_STD = preprocessing.scale(Icfes[key]["df"].PUNT_INGLES),
            PUNT_CIENCIAS_SOCIALES_STD = preprocessing.scale(Icfes[key]["df"].PUNT_CIENCIAS_SOCIALES),
            PUNT_FISICA_STD = preprocessing.scale(Icfes[key]["df"].PUNT_FISICA),
            PUNT_QUIMICA_STD = preprocessing.scale(Icfes[key]["df"].PUNT_QUIMICA),
            PUNT_BIOLOGIA_STD = preprocessing.scale(Icfes[key]["df"].PUNT_BIOLOGIA),
            PUNT_FILOSOFIA_STD = preprocessing.scale(Icfes[key]["df"].PUNT_FILOSOFIA) )
        
        Icfes[key]["df"] = Icfes[key]["df"].assign(
            PUNT_GLOBAL_100 = (Icfes[key]["df"].PUNT_QUIMICA+Icfes[key]["df"].PUNT_BIOLOGIA+
                               Icfes[key]["df"].PUNT_FISICA+(Icfes[key]["df"].PUNT_CIENCIAS_SOCIALES*2)+
                               Icfes[key]["df"].PUNT_FILOSOFIA+(Icfes[key]["df"].PUNT_LENGUAJE*3)+
                               (Icfes[key]["df"].PUNT_MATEMATICAS*3)+Icfes[key]["df"].PUNT_INGLES)/13,
            PUNT_GLOBAL_500 = ((Icfes[key]["df"].PUNT_QUIMICA+Icfes[key]["df"].PUNT_BIOLOGIA+
                                Icfes[key]["df"].PUNT_FISICA+(Icfes[key]["df"].PUNT_CIENCIAS_SOCIALES*2)+
                                Icfes[key]["df"].PUNT_FILOSOFIA+(Icfes[key]["df"].PUNT_LENGUAJE*3)+
                                (Icfes[key]["df"].PUNT_MATEMATICAS*3)+Icfes[key]["df"].PUNT_INGLES)/13)*5,
            PUNT_GLOBAL_STD = (Icfes[key]["df"].PUNT_QUIMICA_STD+Icfes[key]["df"].PUNT_BIOLOGIA_STD+
                               Icfes[key]["df"].PUNT_FISICA_STD+(Icfes[key]["df"].PUNT_CIENCIAS_SOCIALES_STD*2)+
                               Icfes[key]["df"].PUNT_FILOSOFIA_STD+(Icfes[key]["df"].PUNT_LENGUAJE_STD*3)+
                               (Icfes[key]["df"].PUNT_MATEMATICAS_STD*3)+Icfes[key]["df"].PUNT_INGLES_STD)/13,
            AÑO = Icfes[key]["AÑO"] )
        
    else:
        Icfes[key]["df"] =  pd.read_csv(
            Icfes[key]["dir"],delimiter=';',
            usecols= ["COLE_DEPTO_UBICACION","COLE_COD_DEPTO_UBICACION","COLE_MCPIO_UBICACION",
                      "ESTU_COD_RESIDE_MCPIO",'PUNT_GLOBAL','PUNT_INGLES','COLE_NATURALEZA','COLE_JORNADA',
                      'COLE_AREA_UBICACION','COLE_COD_MCPIO_UBICACION'],
            dtype = {"COLE_DEPTO_UBICACION":"str","COLE_MCPIO_UBICACION":"str",'PUNT_GLOBAL':"str",
                     'PUNT_INGLES':"str",'COLE_NATURALEZA':"str",'COLE_JORNADA':"str",
                     'COLE_AREA_UBICACION':'str'},low_memory=False)

        Icfes[key]["df"].columns = list(map(str.upper,Icfes[key]["df"].columns))
        
        Icfes[key]["df"].rename(columns = {'COLE_DEPTO_UBICACION':'DEPTO','COLE_MCPIO_UBICACION':'MUNICIPIO',
                                           "COLE_COD_DEPTO_UBICACION":'COD_DEPTO',
                                           'ESTU_COD_RESIDE_MCPIO':'COD_MUN_ESTU',
                                           'COLE_COD_MCPIO_UBICACION':'COD_MUN_COLE'}, inplace = True)
        
        Icfes[key]["df"] = Icfes[key]["df"].assign(
            PUNT_GLOBAL_500 = pd.to_numeric(Icfes[key]["df"].PUNT_GLOBAL.str.replace(',', '.')),
            PUNT_INGLES = pd.to_numeric(Icfes[key]["df"].PUNT_INGLES.str.replace(',', '.')) )
        
        Icfes[key]["df"] = Icfes[key]["df"].assign(
            PUNT_GLOBAL_STD = preprocessing.scale(Icfes[key]["df"].PUNT_GLOBAL_500),
            PUNT_GLOBAL_100 = Icfes[key]["df"].PUNT_GLOBAL_500/5,
            PUNT_INGLES_STD = preprocessing.scale(Icfes[key]["df"].PUNT_INGLES) )
        
        Icfes[key]["df"] = Icfes[key]["df"].assign(AÑO = Icfes[key]["AÑO"])
        
        
    
        
    conditions = [
        (Icfes[key]["df"]["COD_MUN_COLE"]==11001) | (Icfes[key]["df"]["COD_MUN_ESTU"]==11001),
        (Icfes[key]["df"]["COD_MUN_COLE"]==5001)  | (Icfes[key]["df"]["COD_MUN_ESTU"]==5001 ),
        (Icfes[key]["df"]["COD_MUN_COLE"]==76001) | (Icfes[key]["df"]["COD_MUN_ESTU"]==76001),
        (Icfes[key]["df"]["COD_MUN_COLE"]==8001)  | (Icfes[key]["df"]["COD_MUN_ESTU"]==8001 ),
        (Icfes[key]["df"]["COD_MUN_COLE"]==13001) | (Icfes[key]["df"]["COD_MUN_ESTU"]==13001),
        (Icfes[key]["df"]["COD_MUN_COLE"]==44001) | (Icfes[key]["df"]["COD_MUN_ESTU"]==44001),
        (Icfes[key]["df"]["COD_MUN_COLE"]==23001) | (Icfes[key]["df"]["COD_MUN_ESTU"]==23001),
        (Icfes[key]["df"]["COD_MUN_COLE"]==47001) | (Icfes[key]["df"]["COD_MUN_ESTU"]==47001),
        (Icfes[key]["df"]["COD_MUN_COLE"]==20001) | (Icfes[key]["df"]["COD_MUN_ESTU"]==20001),
        (Icfes[key]["df"]["COD_MUN_COLE"]==70001) | (Icfes[key]["df"]["COD_MUN_ESTU"]==70001),
        (Icfes[key]["df"]["COD_MUN_COLE"]==68001) | (Icfes[key]["df"]["COD_MUN_ESTU"]==68001),
        (Icfes[key]["df"]["COD_MUN_COLE"]==54001) | (Icfes[key]["df"]["COD_MUN_ESTU"]==54001),
        (Icfes[key]["df"]["COD_MUN_COLE"]==73001) | (Icfes[key]["df"]["COD_MUN_ESTU"]==73001),
        (Icfes[key]["df"]["COD_MUN_COLE"]==50001) | (Icfes[key]["df"]["COD_MUN_ESTU"]==50001),
        (Icfes[key]["df"]["COD_MUN_COLE"]==66001) | (Icfes[key]["df"]["COD_MUN_ESTU"]==66001),
        (Icfes[key]["df"]["COD_MUN_COLE"]==17001) | (Icfes[key]["df"]["COD_MUN_ESTU"]==17001),
        (Icfes[key]["df"]["COD_MUN_COLE"]==52001) | (Icfes[key]["df"]["COD_MUN_ESTU"]==52001) ]
    
    choices = [11001,5001,76001,8001,13001,44001,23001,47001,20001,
               70001,68001,54001,73001,50001,66001,17001,52001 ]
    
    Icfes[key]["df"] = Icfes[key]["df"].assign( COD_MUN=np.select(conditions, choices) )
    
    Icfes[key]["df"] = Icfes[key]["df"].loc[
        (Icfes[key]["df"]['COD_MUN'] == 11001) | (Icfes[key]["df"]['COD_MUN'] == 5001)  | 
        (Icfes[key]["df"]['COD_MUN'] == 76001) | (Icfes[key]["df"]['COD_MUN'] == 8001)  | 
        (Icfes[key]["df"]['COD_MUN'] == 13001) | (Icfes[key]["df"]['COD_MUN'] == 44001) | 
        (Icfes[key]["df"]['COD_MUN'] == 23001) | (Icfes[key]["df"]['COD_MUN'] == 47001) | 
        (Icfes[key]["df"]['COD_MUN'] == 20001) | (Icfes[key]["df"]['COD_MUN'] == 70001) | 
        (Icfes[key]["df"]['COD_MUN'] == 68001) | (Icfes[key]["df"]['COD_MUN'] == 54001) | 
        (Icfes[key]["df"]['COD_MUN'] == 73001) | (Icfes[key]["df"]['COD_MUN'] == 50001) | 
        (Icfes[key]["df"]['COD_MUN'] == 66001) | (Icfes[key]["df"]['COD_MUN'] == 17001) | 
        (Icfes[key]["df"]['COD_MUN'] == 52001) ]
                                            
    Icfes[key]["df"] = Icfes[key]["df"].loc[
        (Icfes[key]["df"]["COLE_JORNADA"]!="SABATINA") & (Icfes[key]["df"]["COLE_JORNADA"]!="NOCHE")] 
    
    Icfes[key]["df"] = Icfes[key]["df"][['AÑO',"DEPTO","COD_DEPTO","MUNICIPIO","COD_MUN","PUNT_GLOBAL_500",
                                         'PUNT_GLOBAL_100','PUNT_GLOBAL_STD',"PUNT_INGLES",'PUNT_INGLES_STD',
                                         'COLE_NATURALEZA']]


    
    print(key)



p2011_1
p2011_2
p2012_1
p2012_2
p2013_1
p2013_2
p2014_1
p2014_2
p2015_1
p2015_2
p2016_1
p2016_2
p2017_1
p2017_2
p2018_1
p2018_2
p2019
p2020_1
p2020_2
p2021_1
p2021_2


Se juntan los datos de cada año, y se agrupan por tipo de ciudades

In [23]:
ICFES=pd.concat(
    [Icfes["p2011_1"]["df"],Icfes["p2011_2"]["df"],
     Icfes["p2012_1"]["df"],Icfes["p2012_2"]["df"],
     Icfes["p2013_1"]["df"],Icfes["p2013_2"]["df"],
     Icfes["p2014_1"]["df"],Icfes["p2014_2"]["df"],
     Icfes["p2015_1"]["df"],Icfes["p2015_2"]["df"],
     Icfes["p2016_1"]["df"],Icfes["p2016_2"]["df"],
     Icfes["p2017_1"]["df"],Icfes["p2017_2"]["df"],
     Icfes["p2018_1"]["df"],Icfes["p2018_2"]["df"],
     Icfes["p2019"]["df"],
     Icfes["p2020_1"]["df"],Icfes["p2020_2"]["df"],
     Icfes["p2021_1"]["df"],Icfes["p2021_2"]["df"]
    ])

VariablesICFES=['PUNT_GLOBAL_500','PUNT_GLOBAL_100','PUNT_GLOBAL_STD','PUNT_INGLES','PUNT_INGLES_STD']

Promedio ponderado

Se realiza el puntaje promedio por grupo de ciudades y año para sacar la serie, el sector en el que esté el colegio <br>
Se exportan a Excel

In [25]:
ICFES_Total = ICFES.groupby(['COD_MUN','AÑO']).mean()
ICFES_Total = ICFES_Total.reset_index()                 
ICFES_Total = ICFES_Total.merge(MEN_Poblacion_5_16, on=['COD_MUN','AÑO'])

conditions = [
    (ICFES_Total['COD_MUN'] == 11001) | (ICFES_Total['COD_MUN'] == 5001) | (ICFES_Total['COD_MUN'] == 76001),
    (ICFES_Total['COD_MUN'] == 8001),
    (ICFES_Total['COD_MUN'] == 13001) | (ICFES_Total['COD_MUN'] == 44001) | (ICFES_Total['COD_MUN'] == 23001) |
    (ICFES_Total['COD_MUN'] == 47001) | (ICFES_Total['COD_MUN'] == 20001) | (ICFES_Total['COD_MUN'] == 70001),
    (ICFES_Total['COD_MUN'] == 68001) | (ICFES_Total['COD_MUN'] == 54001) | (ICFES_Total['COD_MUN'] == 73001) | 
    (ICFES_Total['COD_MUN'] == 50001) | (ICFES_Total['COD_MUN'] == 66001) | (ICFES_Total['COD_MUN'] == 17001) | 
    (ICFES_Total['COD_MUN'] == 52001)
             ]

choices = ['PRINCIPALES', 'BARRANQUILLA', 'CARIBE','OTRAS']
ICFES_Total["GRUPO"]= np.select(conditions, choices)

ICFES_Total_grupos=ICFES_Total.groupby(['GRUPO','AÑO']).apply(
    lambda x: pd.Series(np.average(x[VariablesICFES], weights=x["POBLACIÓN_5_16"], axis=0),VariablesICFES))  


In [26]:
ICFES_SECTOR = ICFES.groupby(['COD_MUN','AÑO','COLE_NATURALEZA']).mean()
ICFES_SECTOR = ICFES_SECTOR.reset_index()                 
ICFES_SECTOR = ICFES_SECTOR.merge(MEN_Poblacion_5_16, on=['COD_MUN','AÑO'])

conditions = [
    (ICFES_SECTOR['COD_MUN'] == 11001) | (ICFES_SECTOR['COD_MUN'] == 5001) | (ICFES_SECTOR['COD_MUN'] == 76001),
    (ICFES_SECTOR['COD_MUN'] == 8001),
    (ICFES_SECTOR['COD_MUN'] == 13001) | (ICFES_SECTOR['COD_MUN'] == 44001) | (ICFES_SECTOR['COD_MUN'] == 23001) |
    (ICFES_SECTOR['COD_MUN'] == 47001) | (ICFES_SECTOR['COD_MUN'] == 20001) | (ICFES_SECTOR['COD_MUN'] == 70001),
    (ICFES_SECTOR['COD_MUN'] == 68001) | (ICFES_SECTOR['COD_MUN'] == 54001) | (ICFES_SECTOR['COD_MUN'] == 73001) | 
    (ICFES_SECTOR['COD_MUN'] == 50001) | (ICFES_SECTOR['COD_MUN'] == 66001) | (ICFES_SECTOR['COD_MUN'] == 17001) | 
    (ICFES_SECTOR['COD_MUN'] == 52001)
             ]

choices = ['PRINCIPALES', 'BARRANQUILLA', 'CARIBE','OTRAS']
ICFES_SECTOR["GRUPO"]= np.select(conditions, choices)

ICFES_SECTOR_grupos=ICFES_SECTOR.groupby(['COLE_NATURALEZA','GRUPO','AÑO']).apply(
    lambda x: pd.Series(np.average(x[VariablesICFES], weights=x["POBLACIÓN_5_16"], axis=0),VariablesICFES))  


In [27]:
ICFES_SECTOR=ICFES_SECTOR.sort_values(by=['COLE_NATURALEZA','GRUPO','COD_MUN','AÑO'])
ICFES_Total=ICFES_Total.sort_values(by=['GRUPO','COD_MUN','AÑO'])


with pd.ExcelWriter("ICFES_.xlsx") as writer:
    
    ICFES_Total_grupos.to_excel(writer, sheet_name='Total_grupos')
    ICFES_Total.to_excel(writer, sheet_name='Total_Municipios')
    ICFES_SECTOR_grupos.to_excel(writer, sheet_name='SECTOR_grupos')
    ICFES_SECTOR.to_excel(writer, sheet_name='SECTOR_Municipios')
